In [1]:
import pandas as pd
import json
from pandas import json_normalize 
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
import copy
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
venues = pd.read_json('singapore_geo_dataset.json')
venues.head()

,venue_id,latitude,longitude,venue_name,platform_listing_url,region,neighborhood,address,phone,website,...,review_count,average_rating,review_sample,opening_hours,busy_times,venue_amenities,platform_category,venue_main_image,venue_segment,venue_subsegment
0,ChIJxQ08Bag92jERe0_5mhoBqxs,1.375928,103.955003,HaveFun Karaoke & Live Music Bar (Downtown East),https://www.google.com/maps/place/?q=place_id:...,East Region,Pasir Ris,"1 Pasir Ris Cl, #01-311/312, Singapore 519599",+65 6243 6508,http://www.havefunkaraoke.com/,...,101.0,3.7,{},"{'tuesday': ['11:00:00', '01:00:00'], 'wednesd...",{},{},Karaoke bar,https://storage.googleapis.com/synthesis_geos_...,Entertainment,Entertainment
1,ChIJ47eMgaYQ2jERFX21pRHSOY8,1.331406,103.806817,The Oak Room,https://www.google.com/maps/place/?q=place_id:...,Central Region,Bukit Timah,"18 Greenwood Ave, Singapore 289212",+65 6314 3170,None,...,NaN,NaN,{},{},{},"{'offerings': ['alcohol'], 'types_of_alcohol':...",Pub,https://storage.googleapis.com/synthesis_geos_...,Bar,Bar
2,ChIJh_VG84QX2jER5bM8A0V6ma4,1.333001,103.895961,UB3 Bistro,https://www.google.com/maps/place/?q=place_id:...,Central Region,Geylang,"51 Ubi Ave 3, Singapore 408858",+65 9654 4034,http://www.ub3.sg/,...,128.0,4.8,{},"{'tuesday': ['17:00:00', '22:30:00'], 'wednesd...",{},{},Bistro,https://storage.googleapis.com/synthesis_geos_...,Restaurant,Restaurant
3,ChIJC-992qkZ2jERzMnlGfVSJow,1.303198,103.832124,Nam Sing Hokkien Mee,https://www.google.com/maps/place/?q=place_id:...,Central Region,Orchard,"437 Orchard Rd, Singapore 238878",None,None,...,13.0,1.2,{},{},{},{},Hawker stall,https://storage.googleapis.com/synthesis_geos_...,Hawker Food,Hawker Food
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),https://www.google.com/maps/place/?q=place_id:...,Central Region,Downtown Core,"15 Stamford Rd, #01-83, Arcade @ The Capitol K...",+65 6513 0035,http://www.hopscotch.sg/,...,874.0,4.6,"[{'rating': 5, 'description': 'Food is good as...","{'tuesday': ['12:00:00', '14:30:00'], 'wednesd...","{'friday': {'06:00:00': 0, '07:00:00': 0, '08:...","{'accessibility': ['wheelchairaccessible'], 'a...",Restaurant,https://storage.googleapis.com/synthesis_geos_...,Restaurant,Restaurant


In [3]:
venues.columns

Index(['venue_id', 'latitude', 'longitude', 'venue_name',
       'platform_listing_url', 'region', 'neighborhood', 'address', 'phone',
       'website', 'price_point_bucket', 'review_count', 'average_rating',
       'review_sample', 'opening_hours', 'busy_times', 'venue_amenities',
       'platform_category', 'venue_main_image', 'venue_segment',
       'venue_subsegment'],
      dtype='object')

In [4]:
#Checking out categorical data (Credit: https://dataknowsall.com/blog/eda.html)
cat_data = venues.select_dtypes(include=['object'])
# show counts values of each categorical variable
for colname in cat_data.columns:
    print (colname)
    print (cat_data[colname].value_counts(), '\n')

venue_id
ChIJxQ08Bag92jERe0_5mhoBqxs    1
ChIJcbFPEFIX2jERtVFMUFtv4R8    1
ChIJL2Y8oAgZ2jERvnZhGrcKJJ0    1
ChIJ58zgd34b2jERDEawXFNpWOE    1
ChIJWxvpXnI92jERnSveDSLiiZU    1
                              ..
ChIJRW3GpAgG2jERjcD97jht7uQ    1
ChIJG-xjZ3UQ2jER__adp7zsEsM    1
ChIJH5zGZxoX2jERXoXkO8aQPf8    1
ChIJzdeJJKUZ2jERAWODFauhm-s    1
ChIJwSEc38oX2jERi8JrxEosqL0    1
Name: venue_id, Length: 11865, dtype: int64 

venue_name
Starbucks                                      72
Subway                                         37
KFC                                            35
Ya Kun Kaya Toast                              34
Old Chang Kee                                  32
                                               ..
Boon Keng Bread (文庆面包)                          1
Lao Yao Yao Mi Xian                             1
Ya Chen coffee and beer 亞陳                      1
Alaturka Mediterranean & Turkish Restaurant     1
Goldhill Family Restaurant                      1
Name: venue_name, Len

In [5]:
# Check for null
venues.isnull().sum()

venue_id                   0
latitude                   0
longitude                  0
venue_name                 0
platform_listing_url       0
region                     0
neighborhood               0
address                   39
phone                   3449
website                 4384
price_point_bucket      7493
review_count             638
average_rating           638
review_sample              0
opening_hours              0
busy_times                 0
venue_amenities            0
platform_category          8
venue_main_image           0
venue_segment              9
venue_subsegment           9
dtype: int64

In [6]:
print(venues.platform_category.unique())

['Karaoke bar' 'Pub' 'Bistro' 'Hawker stall' 'Restaurant' 'Bakery'
 'Western restaurant' 'Cafe' 'Seafood restaurant' 'Izakaya restaurant'
 'Bar' 'North Indian restaurant' 'Thai restaurant' 'Japanese restaurant'
 'Sichuan restaurant' 'Chinese restaurant' 'Hamburger restaurant'
 'Hawker centre' 'Cocktail bar' 'Family restaurant' 'Yakitori restaurant'
 'Beer garden' 'Italian restaurant' 'Food court' 'Hotel' 'Bar & grill'
 'Live music venue' 'Sports bar' 'Pizza restaurant' 'Wine store'
 'Ramen restaurant' 'Noodle shop' 'Vietnamese restaurant' 'Liquor store'
 'Indian restaurant' 'Night club' 'Indian Muslim restaurant' 'Disco club'
 'American restaurant' 'Singaporean restaurant' 'Sushi restaurant'
 'Buffet restaurant' 'Halal restaurant' 'Hot pot restaurant'
 'Korean barbecue restaurant' 'Indonesian restaurant' 'Spanish restaurant'
 'Resort hotel' 'Brewery' 'Japanese curry restaurant'
 'Barbecue restaurant' 'Fusion restaurant' 'Yakiniku restaurant'
 'Takeout Restaurant' 'Dim sum restaurant' '

In [7]:
venues.venue_segment.unique()

array(['Entertainment', 'Bar', 'Restaurant', 'Hawker Food',
       'Non Alcoholic', 'Cafe', 'Hotel', 'Stores', None], dtype=object)

In [8]:
venues.venue_subsegment.unique()

array(['Entertainment', 'Bar', 'Restaurant', 'Hawker Food',
       'Non Alcoholic', 'Western', 'Cafe', 'Asian', 'Hotel', 'Alcohol',
       'Halal', 'Vegetarian', 'Fast Food', 'Fine Dining', 'Stores', None],
      dtype=object)

In [9]:
venues.price_point_bucket.unique()

array([None, '$$$$', '$$', '$', '$$$'], dtype=object)

In [10]:
venues.platform_category.unique()

array(['Karaoke bar', 'Pub', 'Bistro', 'Hawker stall', 'Restaurant',
       'Bakery', 'Western restaurant', 'Cafe', 'Seafood restaurant',
       'Izakaya restaurant', 'Bar', 'North Indian restaurant',
       'Thai restaurant', 'Japanese restaurant', 'Sichuan restaurant',
       'Chinese restaurant', 'Hamburger restaurant', 'Hawker centre',
       'Cocktail bar', 'Family restaurant', 'Yakitori restaurant',
       'Beer garden', 'Italian restaurant', 'Food court', 'Hotel',
       'Bar & grill', 'Live music venue', 'Sports bar',
       'Pizza restaurant', 'Wine store', 'Ramen restaurant',
       'Noodle shop', 'Vietnamese restaurant', 'Liquor store',
       'Indian restaurant', 'Night club', 'Indian Muslim restaurant',
       'Disco club', 'American restaurant', 'Singaporean restaurant',
       'Sushi restaurant', 'Buffet restaurant', 'Halal restaurant',
       'Hot pot restaurant', 'Korean barbecue restaurant',
       'Indonesian restaurant', 'Spanish restaurant', 'Resort hotel',
       

In [11]:
venues.average_rating.unique()

array([3.7, nan, 4.8, 1.2, 4.6, 4.1, 4. , 3.3, 4.4, 3.5, 5. , 4.2, 4.5,
       2.5, 3.9, 4.9, 3.8, 4.3, 3.6, 2.8, 3. , 4.7, 3.1, 3.2, 2.7, 1.7,
       3.4, 2. , 2.3, 1. , 1.8, 2.9, 1.9, 2.4, 2.1, 2.6, 2.2, 1.3, 1.5,
       1.6, 1.4])

# Data Cleaning
Dropping columns as they are not useful information for the analysis:
phone, website, review_sample, venue_main_image


In [12]:
venues_updated = venues.drop(['address','platform_listing_url', 'review_sample', 'venue_main_image'], axis=1)
venues_updated = pd.concat([venues_updated.drop(['venue_amenities'], axis=1), pd.json_normalize(venues_updated['venue_amenities'])], axis=1)
col_to_replace = ['phone','website']
venues_updated[col_to_replace] = venues_updated[col_to_replace].notna().astype(int)
venues_updated['review_count'].fillna(0, inplace=True)
venues_updated.head()


,venue_id,latitude,longitude,venue_name,region,neighborhood,phone,website,price_point_bucket,review_count,...,crowd,dining_options,food_options,happy_hour,highlights,payments,planning,ordering_options,amenities,popular_for
0,ChIJxQ08Bag92jERe0_5mhoBqxs,1.375928,103.955003,HaveFun Karaoke & Live Music Bar (Downtown East),East Region,Pasir Ris,1,1,None,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ChIJ47eMgaYQ2jERFX21pRHSOY8,1.331406,103.806817,The Oak Room,Central Region,Bukit Timah,1,0,None,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ChIJh_VG84QX2jER5bM8A0V6ma4,1.333001,103.895961,UB3 Bistro,Central Region,Geylang,1,1,None,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ChIJC-992qkZ2jERzMnlGfVSJow,1.303198,103.832124,Nam Sing Hokkien Mee,Central Region,Orchard,0,0,None,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),Central Region,Downtown Core,1,1,$$$$,874.0,...,"[groups, familyfriendly]",[seating],[vegetarian_options],[happyhour_drinks],[great_cocktails],[card],[accepts_reservations],NaN,NaN,NaN


In [13]:
venues_updated.shape

(11865, 30)

In [14]:
venues_updated.columns

Index(['venue_id', 'latitude', 'longitude', 'venue_name', 'region',
       'neighborhood', 'phone', 'website', 'price_point_bucket',
       'review_count', 'average_rating', 'opening_hours', 'busy_times',
       'platform_category', 'venue_segment', 'venue_subsegment', 'offerings',
       'types_of_alcohol', 'accessibility', 'atmosphere', 'crowd',
       'dining_options', 'food_options', 'happy_hour', 'highlights',
       'payments', 'planning', 'ordering_options', 'amenities', 'popular_for'],
      dtype='object')

# Part 1: Size of opportunity in Singapore
In order to gauge the size of opportunity in Singapore, we should look at different market sizes available to Boozeless based on the criteria that Boozeless has identified.

Also, when estimating the size of market, there needs to be a range to ndicate the bottom line and the top line.

We can measure opportunities at different levels: Total Addressable Market (TAM) and Serviceable Addressable Market (SAM)

Definition for TAM: Total market demand for a product or service
1) Remove from sub-segment: 'Stores'
2) High-end consumers - Remove from sub-segment : 'Fast Food', 'Hawker Food'

Definition for SAM: Focus on Specialisations and criteria: 
1) Young, Mild and Free (modern venues connected with an active, healthy and sporty lifestyle) - these restaurants relate to physical activity (Eg. Vegetarian options, presence of highlights) 
Restaurants with highlights, vegetarian_options/organic_food/ vegan_options under food_options, vegetarian restaurants under subsegment
2) Aesthetically Anchored (venues with unique design, where people go for special occasions) - looking at places that are conventionally high class 
'Fine Dining' under venue_subsegment, Cocktail Bars, Hotels, 3 - 4 dollar signs in 
3) Haute Cuisine (boundary pushing restaurants that aren't afraid to try new things in their menus) - Cuisines that are known to be risk takers


Total F&B Venues: 11865

TAM: 9276

SAM: 1505

- Crit 1: 738
- Crit 2: 939
- Crit 3: 104


In [15]:
# Calculate TAM
remove_subsegments = ['Stores','Fast Food', 'Hawker Food']
venues_tam = venues_updated[~venues_updated.venue_subsegment.isin(remove_subsegments)]
venues_tam.shape

(9276, 30)

In [16]:
# Calculate SAM
## Young, Mild and Free
venues_sam_1 = venues_tam[(~venues_tam.highlights.isna()) | 
                            (~venues_tam['food_options'].isna())| 
                            (venues_tam.venue_subsegment == 'Vegetarian')]
venues_sam_1.shape
#venues_sam_1.to_csv('venue_sam1.csv')


(738, 30)

In [17]:
## Aesthetically Anchored
venues_sam_2 = venues_tam[(venues_tam.venue_subsegment == 'Hotel')|
                            (venues_tam.venue_subsegment == 'Fine Dining')| 
                            (venues_tam.platform_category == 'Cocktail bar')|
                            (venues_tam.price_point_bucket.isin(['$$$','$$$$']))]
venues_sam_2.shape
#venues_sam_2.to_csv('venue_sam2.csv')

(939, 30)

In [18]:
## Haute Cuisine
venues_sam_3 = venues_tam[(venues_tam.platform_category == 'Fusion restaurant')| 
                            (venues_tam.platform_category == 'French restaurant')|
                            (venues_tam.platform_category == 'Asian fusion restaurant')|
                            (venues_tam.platform_category == 'Gastropub') | 
                            (venues_tam.platform_category == 'Modern French restaurant')]
venues_sam_3.shape
#venues_sam_3.to_csv('venue_sam3.csv')

(104, 30)

In [19]:
venues_sam = pd.concat([venues_sam_1, venues_sam_2, venues_sam_3])
venues_sam = venues_sam.drop_duplicates(subset=['venue_id'], keep='first')
venues_sam

,venue_id,latitude,longitude,venue_name,region,neighborhood,phone,website,price_point_bucket,review_count,...,crowd,dining_options,food_options,happy_hour,highlights,payments,planning,ordering_options,amenities,popular_for
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),Central Region,Downtown Core,1,1,$$$$,874.0,...,"[groups, familyfriendly]",[seating],[vegetarian_options],[happyhour_drinks],[great_cocktails],[card],[accepts_reservations],NaN,NaN,NaN
14,ChIJCwAA0BQZ2jERTLFVUntAH1o,1.273767,103.845040,J Bar,Central Region,Downtown Core,1,1,$$,94.0,...,[groups],[seating],NaN,[happyhour_drinks],"[live_performances, great_cocktails]",[card],[accepts_reservations],NaN,[wifi],NaN
31,ChIJD6QYJTYZ2jERv2FiNwjeSl4,1.278633,103.844331,Levant Bar,Central Region,Outram,1,1,None,460.0,...,[groups],[seating],NaN,"[happyhour_food, happyhour_drinks]","[dancing, great_cocktails]",[card],[accepts_reservations],NaN,NaN,NaN
37,ChIJ7zeih40Z2jERqKuuZ7TxZk8,1.305368,103.832537,Crossroads Cafe,Central Region,Orchard,1,1,$$$,454.0,...,"[familyfriendly, groups]",[seating],[vegetarian_options],[happyhour_drinks],[great_cocktails],[card],NaN,[delivery],[wifi],NaN
38,ChIJ5SCE0q4e2jERLiATjgKeV90,1.245772,103.826783,THE CLIFF,Central Region,Southern Islands,1,1,$$$,807.0,...,"[groups, familyfriendly, lgbtq_friendly]","[private_dining_room, seating]",[vegetarian_options],NaN,NaN,[card],"[accepts_reservations, reservations_required]",NaN,[wifi],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11215,ChIJN4ugsccZ2jERjrPCQWJTqkw,1.309689,103.854422,Mohammadi Restaurant,Central Region,Rochor,0,0,None,163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[delivery],NaN,NaN
11504,ChIJuTpnZVQZ2jEReDI4VP2u_w0,1.306578,103.829612,MERCI MARCEL ORCHARD,Central Region,Orchard,1,1,None,2041.0,...,"[groups, familyfriendly]",[seating],NaN,NaN,NaN,[card],[accepts_reservations],NaN,NaN,NaN
11540,ChIJFUNR1fgZ2jER8_PFL26c8Ao,1.321709,103.825311,The Café & Terrace Restaurant,Central Region,Novena,1,1,None,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11651,ChIJwbAPs20Q2jERZH36LO4sSzc,1.338644,103.755944,Dragon Restaurant Group,West Region,Bukit Batok,1,0,None,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[delivery],NaN,NaN


In [20]:
venues_sam.shape

(1505, 30)

# Part 2: Top 20 - 25 priority venues to be launched
To establish priority, there are a few factors we need to consider. The target Audience fit, the three criteria and the potential reach restaurants may have

A scoring matrix would be used to determine the top 20 venues. 

1. High-end customers - We would be getting rid of restaurants with price point buckets less than 3 dollar sign as product caters to high end consumers. (Not part of the scoring matrix, a hard requirement for focus)
2. 3 criteria (60%, 20% each) - Client's focus in terms of criteria
3. Potential Reach restaurant may have - Factors include review_count, presence of website and phone, delivery (increase in reach through delivery orders), average_rating (40%, 10% each)

In [21]:
# 1. Filter in the high-end consumers
venues_sam_prices = venues_sam[(venues_sam['price_point_bucket'] == '$$$') | (venues_sam['price_point_bucket'] == '$$$$')]
venues_sam_prices.shape
venues_sam_prices.groupby(['price_point_bucket'])['venue_id'].count()

price_point_bucket
$$$     443
$$$$     79
Name: venue_id, dtype: int64

In [22]:
# 3 criteria (60%, 20% each)
venues_scoring = venues_sam_prices.copy(deep = True)
#crit 1 young
young = (~venues_scoring.highlights.isna()) |(~venues_scoring['food_options'].isna())| (venues_scoring.venue_subsegment == 'Vegetarian')
venues_scoring['crit_1'] = young.astype(int)

#crit 2 aesthetically anchored
aesthetic = (venues_scoring.venue_subsegment == 'Hotel')| (venues_scoring.venue_subsegment == 'Fine Dining')|(venues_scoring.platform_category == 'Cocktail bar')|(venues_scoring.price_point_bucket.isin(['$$$','$$$$']))
venues_scoring['crit_2'] = aesthetic.astype(int)

#crit 3 haute cuisine
haute = (venues_scoring.platform_category == 'Fusion restaurant')| (venues_scoring.platform_category == 'French restaurant')|(venues_scoring.platform_category == 'Asian fusion restaurant')|(venues_scoring.platform_category == 'Gastropub') | (venues_scoring.platform_category == 'Modern French restaurant')
venues_scoring['crit_3'] = haute.astype(int)

venues_scoring.head()

,venue_id,latitude,longitude,venue_name,region,neighborhood,phone,website,price_point_bucket,review_count,...,happy_hour,highlights,payments,planning,ordering_options,amenities,popular_for,crit_1,crit_2,crit_3
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),Central Region,Downtown Core,1,1,$$$$,874.0,...,[happyhour_drinks],[great_cocktails],[card],[accepts_reservations],NaN,NaN,NaN,1,1,0
37,ChIJ7zeih40Z2jERqKuuZ7TxZk8,1.305368,103.832537,Crossroads Cafe,Central Region,Orchard,1,1,$$$,454.0,...,[happyhour_drinks],[great_cocktails],[card],NaN,[delivery],[wifi],NaN,1,1,0
38,ChIJ5SCE0q4e2jERLiATjgKeV90,1.245772,103.826783,THE CLIFF,Central Region,Southern Islands,1,1,$$$,807.0,...,NaN,NaN,[card],"[accepts_reservations, reservations_required]",NaN,[wifi],NaN,1,1,0
109,ChIJgbj7RAQZ2jERpqImPoeOUHc,1.282770,103.860319,The Club at Renku,Central Region,Downtown Core,1,1,$$$$,262.0,...,NaN,[great_cocktails],[card],[accepts_reservations],NaN,NaN,NaN,1,1,0
119,ChIJ-6PObBcZ2jERouhn_cAZ3ik,1.279220,103.842153,Chapter 1 Wine Bar & Restaurant,Central Region,Outram,1,1,$$$,37.0,...,NaN,[great_cocktails],[card],[accepts_reservations],[delivery],NaN,NaN,1,1,0


In [23]:
# Other consideration (40%, 10% each)
## Con 1: Presence of Phone Number (5%) - Share with presence of Website
phone = (venues_scoring.phone == 1)
venues_scoring['con_1'] = phone.astype(int)

## Con 2: Presence of Website (5%) - Share with presence of Phone Number
website = (venues_scoring.website == 1)
venues_scoring['con_2'] = website.astype(int)

## Con 3: Delivery 
delivery = (venues_scoring.ordering_options.isna())
venues_scoring['con_3'] = delivery.astype(int)

## Con 4: Review Count 
scaler = MinMaxScaler()
venues_scoring['con_4'] = np.log1p(venues_scoring['review_count'])
venues_scoring['con_4'] = scaler.fit_transform(venues_scoring[['con_4']])

## Con 5: Average Rating
venues_scoring['con_5'] = scaler.fit_transform(venues_scoring[['average_rating']])

venues_scoring.head()

,venue_id,latitude,longitude,venue_name,region,neighborhood,phone,website,price_point_bucket,review_count,...,amenities,popular_for,crit_1,crit_2,crit_3,con_1,con_2,con_3,con_4,con_5
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),Central Region,Downtown Core,1,1,$$$$,874.0,...,NaN,NaN,1,1,0,1,1,1,0.609520,0.809524
37,ChIJ7zeih40Z2jERqKuuZ7TxZk8,1.305368,103.832537,Crossroads Cafe,Central Region,Orchard,1,1,$$$,454.0,...,[wifi],NaN,1,1,0,1,1,0,0.543975,0.428571
38,ChIJ5SCE0q4e2jERLiATjgKeV90,1.245772,103.826783,THE CLIFF,Central Region,Southern Islands,1,1,$$$,807.0,...,[wifi],NaN,1,1,0,1,1,1,0.601535,0.619048
109,ChIJgbj7RAQZ2jERpqImPoeOUHc,1.282770,103.860319,The Club at Renku,Central Region,Downtown Core,1,1,$$$$,262.0,...,NaN,NaN,1,1,0,1,1,1,0.489034,0.761905
119,ChIJ-6PObBcZ2jERouhn_cAZ3ik,1.279220,103.842153,Chapter 1 Wine Bar & Restaurant,Central Region,Outram,1,1,$$$,37.0,...,NaN,NaN,1,1,0,1,1,0,0.295128,0.904762


In [24]:
venues_scoring['total_score'] = 20*(venues_scoring['crit_1']+venues_scoring['crit_2']+venues_scoring['crit_3']) + 5*(venues_scoring['con_1']+venues_scoring['con_2']) + 10*(venues_scoring['con_3']+venues_scoring['con_4']+venues_scoring['con_5'])
venues_scoring = venues_scoring.sort_values(by='total_score', ascending=False)
venues_scoring.head(25).to_csv('second_25.csv')
venues_scoring.head(25)

,venue_id,latitude,longitude,venue_name,region,neighborhood,phone,website,price_point_bucket,review_count,...,popular_for,crit_1,crit_2,crit_3,con_1,con_2,con_3,con_4,con_5,total_score
6504,ChIJ2eBfe3UZ2jERWLXoC909C84,1.286251,103.854258,Saint Pierre,Central Region,Downtown Core,1,1,$$$$,241.0,...,NaN,1,1,1,1,1,1,0.480693,0.857143,93.378356
8454,ChIJsRAjOg0Z2jERznft3YWnoec,1.281301,103.845684,Les Bouchons @ Ann Siang,Central Region,Outram,1,1,$$$,822.0,...,NaN,1,1,1,1,1,1,0.603379,0.714286,93.176646
3972,ChIJSRe-Ag8Z2jER4PBxoDTojVQ,1.283160,103.853398,La Brasserie,Central Region,Downtown Core,1,1,$$$,668.0,...,NaN,1,1,1,1,1,0,0.582614,0.666667,82.492802
6381,ChIJ0Wkqc6UZ2jERmam6mzVinfg,1.296621,103.854914,Gunther's,Central Region,Downtown Core,1,1,$$$$,254.0,...,NaN,1,1,1,1,1,0,0.485938,0.761905,82.478423
10261,ChIJzV8SihEZ2jERWpgRyF2wE1k,1.278172,103.850687,Cook & Brew,Central Region,Downtown Core,1,1,$$$,358.0,...,NaN,1,1,1,1,1,0,0.520223,0.619048,81.392704
3499,ChIJbUjtT6cZ2jERJOB8kHFKljI,1.289707,103.851443,Odette,Central Region,Downtown Core,1,1,$$$$,1091.0,...,NaN,1,1,0,1,1,1,0.631726,0.857143,74.888684
11813,ChIJ36AdBg0Z2jERvFl0QFoqZ0E,1.275032,103.843551,Jigger & Pony,Central Region,Downtown Core,1,1,$$$,1062.0,...,NaN,1,1,0,1,1,1,0.629028,0.857143,74.861706
4,ChIJG0xEFbIZ2jER95y7R-Z8JJs,1.293449,103.851829,Hopscotch (Capitol),Central Region,Downtown Core,1,1,$$$$,874.0,...,NaN,1,1,0,1,1,1,0.609520,0.809524,74.190437
9312,ChIJbyclJKMZ2jER-IyFjzQepNY,1.300785,103.839403,Joie | Vegetarian Casual Fine Dining,Central Region,Orchard,1,1,$$$,1293.0,...,NaN,1,1,0,1,1,1,0.648738,0.761905,74.106425
1841,ChIJr84_RCMa2jERW466IPXbCec,1.303992,103.809860,Dempsey Hill,Central Region,Tanglin,1,1,$$$,2021.0,...,NaN,1,1,0,1,1,1,0.693476,0.714286,74.077620


In [25]:
venues_scoring_25 = venues_scoring[['venue_name','crit_1','crit_2','crit_3','con_1','con_2','con_3','con_4','con_5','total_score']]
venues_scoring_25 = venues_scoring_25.rename(columns={'crit_1':'young','crit_2':'aesthetic','crit_3':'haute','con_1':'phone','con_2':'website','con_3':'delivery','con_4':'review_count','con_5':'avg_rating'})
venues_scoring_25.head(25)

,venue_name,young,aesthetic,haute,phone,website,delivery,review_count,avg_rating,total_score
6504,Saint Pierre,1,1,1,1,1,1,0.480693,0.857143,93.378356
8454,Les Bouchons @ Ann Siang,1,1,1,1,1,1,0.603379,0.714286,93.176646
3972,La Brasserie,1,1,1,1,1,0,0.582614,0.666667,82.492802
6381,Gunther's,1,1,1,1,1,0,0.485938,0.761905,82.478423
10261,Cook & Brew,1,1,1,1,1,0,0.520223,0.619048,81.392704
3499,Odette,1,1,0,1,1,1,0.631726,0.857143,74.888684
11813,Jigger & Pony,1,1,0,1,1,1,0.629028,0.857143,74.861706
4,Hopscotch (Capitol),1,1,0,1,1,1,0.609520,0.809524,74.190437
9312,Joie | Vegetarian Casual Fine Dining,1,1,0,1,1,1,0.648738,0.761905,74.106425
1841,Dempsey Hill,1,1,0,1,1,1,0.693476,0.714286,74.077620


# Part 3: How can publicly available data be used to help inform the launch & marketing activities for their roll-out in F&B outlets in Singapore?
1. Social Media - Social Media data of the F&B outlets can be scrapped to understand how much social media presence the F&B restaurants have. As mentioned, the presence of communication methods to customers, such as websites and phone numbers in the dataset are crucial to understand if the F&B Outlets have the appropriate means to reach customers. Likewise, this applies for the availability of the latest social media accounts, such as Facebook, Instagram and TikTok. Boozeless could leverage on the social media and communication means to get to customers.
2. Use of Market Research tools and trend softwares such as Google Trends to look at what time period is the best for marketing and launch of the product, marketing and sales strategies (Eg. time-based discounts offered, samples to be distributed)
3. Competitor web-scrapping to gain an idea of the spirits culture and crucial information such as price points so that Boozeless can understand the potential price points its competitors are selling at etc.
4. Open Sourced Maps - Publicly available maps such as the provided one that label the spaces of opportunities Boozeless could use for marketing and launch events that may also be close to the F&B outlets as mentioned above.


# Appendix


## ML trial
Goal: To predict the accuracies of data with existing price_point_bucket so that we can fill in the missing values in the column in other rows.
- Post Note tldr: It is possible to predict with relative accuracy whether the outlet fit a high-end consumer regular (I.e. 3 $ and more), around 93.7% accuracy, but due to time constraints, it is not included in the main section. Future alterations include: Using other models to test, Using the model to predict whether a venue is likely to be high-end. More evaluation of the model is required.

### Data Prep
1. Remove 'venue_id','latitude', 'longitude','venue_name', 'review_sample', 'opening_hours', 'busy_times', 
                     'venue_main_image', 'venue_segment','platform_listing_url', 'address', 'phone', 'website'
2. Convert the following to binary if not NaN: 'platform_listing_url', 'address', 'phone', 'website',
3. Do one-hot encoding for 'region', 'neighborhood','platform_category','venue_subsegment'
4. Convert 'average_rating' and 'review_count' to between 0 - 1 using min_max scaling
5. Convert 'price_point_bucket' as labels
6. Expand 'venue_amenities', a column made out of dictionaries and create columns with it using json_normalize, and change the content of values in these columns to 1 if it's not NaN and 0 if it is

In [26]:
# 1. Remove unwanted columns
unwanted_columns = ['venue_id','latitude', 'longitude','venue_name', 'review_sample', 'opening_hours', 'busy_times', 
                     'venue_main_image', 'venue_segment']
venues_processed = venues.drop(unwanted_columns, axis=1)

# 2. Convert specified columns to binary
binary_columns = ['platform_listing_url', 'address', 'phone', 'website']
for col in binary_columns:
    venues_processed[col] = venues_processed[col].notna().astype(int)

# 3. One-hot encoding for categorical columns
categorical_columns = ['neighborhood','region','platform_category', 'venue_subsegment']
venues_processed = pd.get_dummies(venues_processed, columns=categorical_columns)

# 4. Min-max scaling for 'average_rating' and 'review_count'
scaler = MinMaxScaler()
venues_processed[['average_rating', 'review_count']] = scaler.fit_transform(venues_processed[['average_rating', 'review_count']])
venues_processed['review_count'].fillna(0, inplace=True)
venues_processed['review_count_log'] = np.log1p(venues_processed['review_count'])
venues_processed = venues_processed.drop('review_count', axis=1)
venues_processed = venues_processed.dropna(subset=['average_rating'])

# 5. Convert 'price_point_bucket' to labels
venues_processed = venues_processed.dropna(subset=['price_point_bucket'])
venues_processed['price_point_bucket'] = venues_processed['price_point_bucket'].replace({'$$$': 1, '$$$$': 1, '$$': 0,'$': 0})

## 6. 
# Apply json_normalize to expand 'venue_amenities' column
expanded_amenities = pd.json_normalize(venues_processed['venue_amenities'])

# Replace NaN values with 0 and non-NaN values with 1
expanded_amenities = expanded_amenities.notna().astype(int)

venues_processed = venues_processed.reset_index()

# Concatenate the expanded amenities DataFrame with the original DataFrame
venues_processed = pd.concat([venues_processed, expanded_amenities], axis=1)

# Drop the original 'venue_amenities' column
venues_processed = venues_processed.drop('venue_amenities', axis=1)


venues_processed.head()
#expanded_amenities.head()

,index,platform_listing_url,address,phone,website,price_point_bucket,average_rating,neighborhood_Ang Mo Kio,neighborhood_Bedok,neighborhood_Bishan,...,food_options,happy_hour,highlights,offerings,payments,planning,types_of_alcohol,amenities,ordering_options,popular_for
0,4,1,1,1,1,1,0.900,0,0,0,...,1,1,1,1,1,1,1,0,0,0
1,11,1,1,1,1,0,0.775,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14,1,1,1,1,0,0.800,0,0,0,...,0,1,1,1,1,1,1,1,0,0
3,16,1,1,1,1,0,0.675,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,18,1,1,1,1,0,0.850,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## ML Logistic Regression 

In [27]:
# 1.Data Preparation
X = venues_processed.drop(['price_point_bucket'], axis=1)  # Features
y = venues_processed['price_point_bucket']  # Target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# 2.Model Training
# Initialize and train logistic regression model
logistic_regression_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
logistic_regression_model.fit(X_train, y_train)

# 3.Model Evaluation
# Predict on the test set
y_pred = logistic_regression_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.9374045801526718
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1153
           1       0.74      0.74      0.74       157

    accuracy                           0.94      1310
   macro avg       0.85      0.85      0.85      1310
weighted avg       0.94      0.94      0.94      1310



In [28]:
# Accessing coefficients and corresponding features
coefficients = logistic_regression_model.coef_[0]
features = X.columns

# Create a DataFrame to display coefficients and corresponding features
coefficients_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
coefficients_df['Absolute Coefficient'] = coefficients_df['Coefficient'].abs()
coefficients_df = coefficients_df.sort_values(by='Absolute Coefficient', ascending=False)

# Display the DataFrame
print(coefficients_df.head(20))


                                  Feature  Coefficient  Absolute Coefficient
193                venue_subsegment_Hotel     1.956824              1.956824
119               platform_category_Hotel     1.956824              1.956824
19                   neighborhood_Geylang    -1.916242              1.916242
186                  venue_subsegment_Bar    -1.687924              1.687924
2                                 address    -1.535094              1.535094
1                    platform_listing_url    -1.535094              1.535094
18             neighborhood_Downtown Core     1.514588              1.514588
210                      types_of_alcohol     1.435382              1.435382
208                              payments     1.404567              1.404567
23                   neighborhood_Kallang    -1.297624              1.297624
205                            happy_hour    -1.194807              1.194807
209                              planning     1.177990              1.177990